Code pour mettre et récupérer un model sur docker 


In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split



2. Chargement et préparation des données

In [ ]:
from sklearn.preprocessing import LabelEncoder

def load_and_prepare_data(directory_path):
    """Charge les fichiers JSON et prépare les données pour la vectorisation."""
    combined_data = {}
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(directory_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                combined_data.update(data)

    # Extraire les champs pertinents
    texts = [content.get('readme_clean', '') for content in combined_data.values()]
    labels = [content['mainLanguage'] for content in combined_data.values()]

    # Encodage des labels
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(labels)

    return texts, encoded_labels, label_encoder

# Charger et préparer les données
texts, labels, label_encoder = load_and_prepare_data(directory_path)





In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Tokeniser les textes
tokenized_texts = [word_tokenize(text) for text in texts]

# Entraîner le modèle Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)

# Fonction pour obtenir un vecteur moyen par document
def vectorize_documents(tokenized_texts, model):
    """Vectorise une liste de documents tokenisés en utilisant Word2Vec."""
    document_vectors = []
    for tokens in tokenized_texts:
        vectors = [model.wv[word] for word in tokens if word in model.wv]
        if vectors:
            document_vectors.append(sum(vectors) / len(vectors))
        else:
            document_vectors.append([0] * model.vector_size)  # Vecteur nul si aucun mot n'est trouvé
    return document_vectors

# Vectoriser les documents
document_vectors = vectorize_documents(tokenized_texts, word2vec_model)


In [ ]:
# Diviser les données
X_train, X_test, y_train, y_test = train_test_split(document_vectors, labels, test_size=0.2, random_state=42)


3. Modèle et classifieur et Connection


In [ ]:
# Modèle entrainé
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

In [ ]:
# Log le modèle (autrement dit le mettre sur Mlflow Docker)
# Enregistrer les modèles avec un run parent
with mlflow.start_run(run_name="model_training") as parent_run:
    
    with mlflow.start_run(run_name="random_forest", nested=True):
        mlflow.sklearn.log_model(clf, 'model')

In [ ]:
# Exemple pour charger un modèle depuis un run spécifique
run_id = '488f4afcbc1b46729f39b792c87bb08d'  # Remplacez par l'ID du run que vous souhaitez inspecter
logged_model = f'runs:/{run_id}/random_forest_model'

# Charger le modèle
model = mlflow.sklearn.load_model(logged_model)

# Vérifier si le modèle est chargé correctement
print(model)

In [ ]:
# Remplace <RUN_ID> par l'ID du run où est enregistré le modèle qu'on veut charger puis nom de lui
model = mlflow.sklearn.load_model(f"runs:/5fefbe6797a24f0fafc544605ae42970/son_nom")

import numpy as np

test_input = np.random.rand(1, model.n_features_in_)  # Crée un input avec le bon nombre de colonnes
prediction = model.predict(test_input)

print("Prédiction :", prediction)